<a href="https://colab.research.google.com/github/WebiksInc/Stop-Words-Hebrew/blob/main/prepare_stop_word_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stop words in Hebrew

In [13]:
"""
List of stop words in Hebrew produced by Universal Dependencies  of the The Israeli Association of Human Language Technologies (IAHLT)
"""

import numpy as np
import pandas as pd
import requests as req
!pip install pyconll
import pyconll
# data ud train/test/dev
ud_dev = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/master/he_iahltwiki-ud-dev.conllu'
ud_test = 'https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/raw/master/he_iahltwiki-ud-test.conllu'
ud_train = 'https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/blob/master/he_iahltwiki-ud-train.conllu?raw=true'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
#download data
def get_data(url:str,name:str):
  res = req.get(url)
  file = open(name + '.txt', 'w')
  file.write(res.text)
  file.close()
get_data(ud_dev,'ud_dev')
get_data(ud_test,'ud_test')
get_data(ud_train,'ud_train')

In [15]:
#load the UD data
dev = pyconll.load_from_file('/content/ud_dev.txt')
train = pyconll.load_from_file('/content/ud_test.txt')
test = pyconll.load_from_file('/content/ud_train.txt')

In [16]:
def remove_niqqud_from_string(my_string):
    return ''.join(['' if  1456 <= ord(c) <= 1479 else c for c in my_string])

def Extracts_stopwords(dat:pyconll):
  """
  Extracts the stopwords form the UD Schema
  """
  POS_tag  = ['DET','ADP','PRON','CCONJ','SCONJ','SYM']
  lst_words = [token.form for sentence in dat for token in sentence if token.upos in POS_tag]
  return lst_words 

words_plus = {'אגב', 'הואיל' , 'הלא', 'הרי', 'כיאלו', 'כיון','יען','על' 'מנת','ודאי','גם','אפילו','יש','רק'}
words_minus = {'%','&','(','+','-','/',':','=','–','°','בינ','שורת','אודות','לאזור','במקביל','כשב-1948','ד','בתוכ','הו','י','ך','ם','ן',"נ'",'נאומ'}


stopwords_raw = set(Extracts_stopwords(dev) + Extracts_stopwords(train) + Extracts_stopwords(test)) #Extracts the stopwords form the UD Schema
# stopwords_raw = set(map(remove_niqqud_from_string,stopwords_raw)) # Optionally, delete the words with the niqqud
stopwords = stopwords_raw - words_minus # Delete incorrect words
stopwords = stopwords | words_plus #Add missing words that we identified . Note that there is another addition of words below

#Note: The list includes English words that happened to be in the database. For a complete list in English we recommend nltk package.

In [17]:
# Add prepositions. A detailed explanation of the need appears in README

al = ['אלי', 'אליך', 'אליך', 'אליו', 'אליה', 'אלינו', 'אליכם','אליכן', 'אליהם','אליהן']
hal = ['עלי','עליך','עליך','עליו','עליה','עלינו','עליכם','עליכן, עליהם','עליהן']
him = ['עמי','עמך','עמו','עמה','עמנו','עמכם','עמכן','עמם','עמן','עמהם','עמהן']
ath_with = ['אתן','אתם','אתכן','אתכם','אתנו','אתה','אתו','אתך','אתי']
ath = ['אותי','אותך','אותו','אותה','אותנו','אתכם','אתכן','אותם','אותן']
shel = ['שלי','שלך','שלו','שלה','שלנו','שלכם','שלכן','שלהם','שלהן',]
men = ['ממני','ממך','ממנו','ממנה','מכם','מכן','מהם','מהן']
azhel = ['אצלי','אצלך','אצלו','אצלה','אצלנו','אצלכם','אצלכן','אצלם','אצלן']
bshvil = ['בשבילי','בשבילך','בשבילו','בשבילה','בשבילנו','בשבילכם','בשבילכן','בשבילם','בשבילן']
bin = ['ביני','בינך','בינו','בינה','בינינו','ביניכם','ביניכן','ביניהם','ביניהן','בינם','בינן']
lafi = ['לפי','לפיך','לפיו','לפיה','לפינו','לפיכם','לפיכן','לפיהם','לפיהן']
lafni = ['לפני','לפניך','לפניו','לפניה','לפנינו','לפניכם','לפניכן','לפניהם','לפניהן']
chmo = ['כמוני','כמוך','כמוהו','כמוה','כמונו','כמוכם','כמוכן','כמוהם','כמוהן']
lakrt = ['לקראתי','לקראתך','לקראתו','לקראתה','לקראתנו','לקראתכם','לקראתכן','לקראתם','לקראתן']
pepositions = {'אל':al,
               'על':hal,
               'עם':him,
               '(=עם) את':him,
               'את (תוויות מושא ישיר':ath,
               'של' : shel,
               'מן' : men,
               'אצל' : azhel,
               'בשביל' : bshvil,
               'בין' : bin,
               'לפי' : lafi,
               'לפני' : lafni,
               'כמו' : chmo,
               'לקראת' : lakrt
               }
pepositions_lst = [item for sublist in list(pepositions.values()) for item in sublist]

In [18]:
stopwords = stopwords | set(pepositions_lst)
pd.DataFrame({'stopswords' : list(stopwords)}).to_csv('/content/drive/My Drive/Colab Notebooks/stopswords_list.csv',index = False)
print(', '.join([word if index%15 != 0 else '\n,' + word for index,word in  enumerate(stopwords)] ))


,שכן, לפיך, בגלל, עקב, נכון, אותן, עצמן, תחתי, מרבית, תוך, מכם, מסביב, לתוך, מכיוון, כלפי, 
,כלומר, לקראתכם, למן, שלהן, שלה, איזה, כמוכן, אותי, עצמה, מול, עובר, כש, וְ, You, מלבד, 
,הרי, עמן, אנו, The, ְהַ, בינך, עמכם, הגם, פי, כפי, מצד, כמוהם, יותר, בִ, אצלנו, 
,כולו, עד, מש, ביניהם, בשבילו, קרוב, משום, מישהו, עמכן, לכבוד, לבל, לגבי, מהן, של, לא, 
,אנחנו, אשר, אחרי, ללא, למעט, כן, מעט, אותה, כולם, הואיל, אותך, שלו, לפניהן, זאת, כך, 
,פחות, עת, עמם, לפני, כגון, מקרב, ה, שלנו, את, שום, על-פי, לשם, בשבילנו, בשבילכן, אלי, 
,ביני, אחר, לקראתנו, לפיכם, תחת, חרף, כי, בהתבסס, לנוכח, לקראתה, עמה, הלא, גבי, מדי, וַ, 
,אִם, אליכן, עלינו, לציד, במקומ, עצמם, עלמנת, מספר, שלכם, מספיק, a, עמי, למען, אליבא, כולה, 
,כדי, בשבילך, אל, הן, אם, יחד, דרכ, שמא, לרגל, מעין, בניגוד, נגד, בנוגע, to, לפינו, 
,למחרת, לקראת, אות, עצמו, זה, ערב, לפנינו, אליכם, לידי, עַד, הּ, לצד, פר, אלמלא, בינינו, 
,לאורך, מאז, מעלי, יש, באם, כם, לבד, מעל, טרום, ערכ, עליו, אצלי, זו, האם, the, 
,בידי, מציד, יען, אותנו, כנגד, מתחת